In [12]:
pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 2.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 2.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip show scikit-learn

Name: scikit-learn
Version: 1.3.0
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/Cellar/jupyterlab/3.4.7/libexec/lib/python3.10/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [230]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.metrics import fbeta_score, precision_score, recall_score, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [231]:
import pandas as pd
import numpy as np
import ydata_profiling
import joblib

After printing column names into a text editor we can see that there are some leading white spaces in front of some column names.
Remove white space from column names by using skipinitialspace

In [250]:
df = pd.read_csv("data/census.csv", skipinitialspace = True)

Functions

In [233]:
def process_data(
    X, categorical_features=[], label=None, training=True, encoder=None, lb=None
):
    """ Process the data used in the machine learning pipeline.

    Processes the data using one hot encoding for the categorical features and a
    label binarizer for the labels. This can be used in either training or
    inference/validation.

    Note: depending on the type of model used, you may want to add in functionality that
    scales the continuous data.

    Inputs
    ------
    X : pd.DataFrame
        Dataframe containing the features and label. Columns in `categorical_features`
    categorical_features: list[str]
        List containing the names of the categorical features (default=[])
    label : str
        Name of the label column in `X`. If None, then an empty array will be returned
        for y (default=None)
    training : bool
        Indicator if training mode or inference/validation mode.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained sklearn OneHotEncoder, only used if training=False.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained sklearn LabelBinarizer, only used if training=False.

    Returns
    -------
    X : np.array
        Processed data.
    y : np.array
        Processed labels if labeled=True, otherwise empty np.array.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained OneHotEncoder if training is True, otherwise returns the encoder passed
        in.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained LabelBinarizer if training is True, otherwise returns the binarizer
        passed in.
    """
    print("***************inside process_data******************")
    if label is not None:
        y = X[label]
#         print("**********y**************")
#         print(y)
#         print("************************")
        X = X.drop([label], axis=1)
    else:
        y = np.array([])

    X_categorical = X[categorical_features].values
    X_continuous = X.drop(*[categorical_features], axis=1)

    if training is True:
        encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
        lb = LabelBinarizer()
        X_categorical = encoder.fit_transform(X_categorical)
        y = lb.fit_transform(y.values).ravel()
    else:
        X_categorical = encoder.transform(X_categorical)
        try:
            y = lb.transform(y.values).ravel()
        # Catch the case where y is None because we're doing inference.
        except AttributeError:
            pass

    X = np.concatenate([X_continuous, X_categorical], axis=1)
    return X, y, encoder, lb

In [455]:
def train_model(X_train, y_train):
    """
    Trains a machine learning model and returns it.

    Inputs
    ------
    X_train : np.array
        Training data.
    y_train : np.array
        Labels.
    Returns
    -------
    model
        Trained machine learning model.
    """
    print("************inside train_model function")
    
#     using logistic regression fails becase ValueError: 
# Found array with 0 sample(s) (shape=(0, 108)) while a minimum of 1 is required by LogisticRegression.

#     lr = LogisticRegression(max_iter=1000, random_state=23)
#     lr.fit(X_train, y_train)

#  Using random forest
    rfc = RandomForestClassifier(random_state=42)
    param_grid = {
        'n_estimators': [5, 10]
#         'max_features': ['auto', 'sqrt'],
#         'max_depth': [4, 5, 100],
#         'criterion': ['gini', 'entropy']
    }
    
    cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
    cv_rfc.fit(X_train, y_train)

# using  Support Vector Machines

#     clf = 


    return cv_rfc

In [465]:
def compute_model_metrics(y, preds):
    """
    Validates the trained machine learning model using precision, recall, and F1.

    Inputs
    ------
    y : np.array
        Known labels, binarized.
    preds : np.array
        Predicted labels, binarized.
    Returns
    -------
    precision : float
    recall : float
    fbeta : float
    """
    print("************inside compute_model_metrics ****************")
    precision = precision_score(y, preds, zero_division=1)
    recall = recall_score(y, preds, zero_division=1)
    fbeta = fbeta_score(y, preds, beta=1, zero_division=1)
#     print("three")
    return precision, recall, fbeta

In [235]:
def save_model(model):
    joblib.dump(model, 'model/model.pkl')

In [236]:
def inference(model, X):
    """ Run model inferences and return the predictions.

    Inputs
    ------
    model : ???
        Trained machine learning model.
    X : np.array
        Data used for prediction.
    Returns
    -------
    preds : np.array
        Predictions from the model.
    """
    preds = model.predict(X)
    return preds

In [251]:
df.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [252]:
print(list(df.columns.values))

['age', 'workclass', 'fnlgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']


In [253]:
df.columns = df.columns.str.replace('-', '_')
print(list(df.columns.values))

['age', 'workclass', 'fnlgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'salary']


In [254]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlgt           32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [345]:
profile = ydata_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

After printing column names into a text editor we can see that there are some leading white spaces in front of some column names.
Remove white space from column names

In [363]:
# df.rename(columns=lambda x: x.strip(), inplace=True)

In [255]:
df.head(2)

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [242]:
# print(df[['workclass']].to_string(index=False))
print(df['workclass'].values)

['State-gov' 'Self-emp-not-inc' 'Private' ... 'Private' 'Private'
 'Self-emp-inc']


split data

In [510]:
train, test = train_test_split(df, test_size=0.20)
cat_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]

In [507]:
def split_data(df):
    train, test = train_test_split(df, test_size=0.20)
    return train, test 

In [499]:
train

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
8188,43,Private,242488,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,5013,0,40,United-States,<=50K
28291,25,Private,193379,Assoc-acdm,12,Never-married,Craft-repair,Not-in-family,White,Male,0,0,45,United-States,<=50K
25096,21,Private,221157,HS-grad,9,Never-married,Other-service,Own-child,Black,Female,0,0,30,United-States,<=50K
10567,47,Federal-gov,55377,Bachelors,13,Never-married,Adm-clerical,Unmarried,Black,Male,0,0,40,United-States,>50K
9620,49,Private,353824,Some-college,10,Divorced,Prof-specialty,Not-in-family,White,Male,0,0,45,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19481,23,Private,184271,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
32166,42,Local-gov,189956,Bachelors,13,Divorced,Prof-specialty,Unmarried,Black,Female,0,0,30,United-States,<=50K
29915,48,Private,232145,HS-grad,9,Divorced,Machine-op-inspct,Not-in-family,White,Male,0,0,40,United-States,<=50K
5146,28,Private,56179,Some-college,10,Never-married,Exec-managerial,Not-in-family,White,Male,2174,0,55,United-States,<=50K


In [511]:
train

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
28687,58,Self-emp-not-inc,105592,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,>50K
20357,35,Private,73715,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
24335,37,Private,167613,HS-grad,9,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,United-States,<=50K
15030,32,Private,193285,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
26772,28,Private,412149,10th,6,Never-married,Farming-fishing,Other-relative,White,Male,0,0,35,Mexico,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23443,42,Local-gov,193537,7th-8th,4,Married-spouse-absent,Other-service,Not-in-family,White,Female,0,0,35,Puerto-Rico,<=50K
22588,36,Private,207568,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States,<=50K
13571,34,Private,186824,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,70,United-States,<=50K
8294,50,Federal-gov,183611,Some-college,10,Never-married,Adm-clerical,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [509]:
x = split_data(df)
type(x)
x[0]

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
16518,49,Private,236586,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,>50K
6047,42,Private,125461,Bachelors,13,Never-married,Sales,Unmarried,White,Male,0,0,40,United-States,<=50K
28494,41,Private,172828,9th,5,Married-civ-spouse,Other-service,Husband,White,Male,0,0,55,Outlying-US(Guam-USVI-etc),<=50K
26366,54,State-gov,198741,Some-college,10,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
7035,51,Private,216475,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,1564,43,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13148,49,Self-emp-inc,127111,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
30055,40,Self-emp-inc,132222,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,<=50K
10705,44,Private,252202,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
7791,19,Private,165310,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Other-relative,White,Male,0,0,20,United-States,<=50K


In [204]:
print(type(train))


print(type(X_train))
print(type(y_train))


print(type(y_test))


print(train.shape)
print(X_train.shape)
print(y_train.shape)


print("----")
print(test.shape)
print(X_test.shape)
print(y_test.shape)
print(type(test))
print(type(X_test))
print(type(y_test))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(26048, 15)
(26048, 107)
(26048,)
----
(6513, 15)
(6513, 107)
(6513,)
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [431]:
test[:2]

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
29464,52,Local-gov,236497,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,<=50K
2490,36,Private,104772,HS-grad,9,Divorced,Craft-repair,Unmarried,White,Male,0,0,48,United-States,<=50K


In [257]:
X_train, y_train, encoder, lb = process_data(
    train, categorical_features=cat_features, label="salary", training=True
)

***************inside process_data******************


KeyError: "['marital-status', 'native-country'] not in index"

In [17]:
# X_train.describe()
# print(type(X_train))

In [18]:
# print(type(X_train))

In [19]:
# X_train[1:2]

In [20]:
# y_train[:2]

In [21]:
# y_train

In [22]:
# print(type(encoder))
# print(encoder)
# print(lb)

Proces the test data with the process_data function.
encoder OneHotEncoder, only used if training=False.

In [23]:
# lb = LabelBinarizer()

In [24]:
# def process_data(
# #     X, categorical_features, label, training, encoder, lb):
#      X, categorical_features=[], label=None, training=True, encoder=None, lb=None):
#     if label is not None:
#         y = X[label]
#         X = X.drop([label], axis=1)
#     else:
#         y = np.array([])

#     X_categorical = X[categorical_features].values
#     X_continuous = X.drop(*[categorical_features], axis=1)

#     if training is True:
#         encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
#         lb = LabelBinarizer()
#         X_categorical = encoder.fit_transform(X_categorical)
#         y = lb.fit_transform(y.values).ravel()
#     else:
#         print("one")
#         X_categorical = encoder.transform(X_categorical)
#         print("two")
#         try:
#             y = lb.transform(y.values).ravel()
#             print("one")
#             print(y)
#         # Catch the case where y is None because we're doing inference.
#         except AttributeError:
#             pass
#     print(y)

# #     X = np.concatenate([X_continuous, X_categorical], axis=1)
#     return X, y, encoder, lb

In [25]:
# X_train, y_train, encoder, lb = process_data(
#     train, categorical_features=cat_features, label="salary", training=True
# )

In [26]:
# print(type(X_train))

In [ ]:
X_test, y_test, encoder, lb = process_data(
    test, categorical_features=cat_features, label='salary', training=False, encoder=encoder, lb=lb
)

In [28]:
# print(type(X_test))

In [29]:
# X_test[1:2]

In [30]:
# print(type(X_test))
# print(type(y_test))
# print(y_test[:5])
# print(len(y_test))
# print(y_test[:-5])

In [31]:
# yTest = y_test.to_numpy() 
# print(type(X_train))
# print(type(y_train))

Train and save a model

In [32]:
lr = train_model(X_train, y_train)

NameError: name 'train_model' is not defined

In [385]:
save_model(lr)

In [386]:
# print(type(lr))

In [387]:
# print(type(X_test))

In [388]:
# print(y_test[:5])

In [389]:
preds = inference(lr, X_test)

In [390]:
# type(preds)

In [391]:
# type(y_test)

In [392]:
# print(type(y_test))
# print(classification_report(y_test, lr.predict(X_test)))

In [393]:
compute_model_metrics(y_test, preds)

(0.705574912891986, 0.25845564773452456, 0.37832788416627744)

In [401]:
def slice_dataframe(df, feature):
    '''
    Function to compute model metrics on slices of the dataset
    '''

    for cls in df[feature].unique():
        print(cls)
        print(type(cls))
        # df_temp = df[df["class"] == cls]
        # mean = df_temp[feature].mean()
        # stddev = df_temp[feature].std()
        # print(f"Class: {cls}")
        # print(f"{feature} mean: {mean:.4f}")
        # print(f"{feature} stddev: {stddev:.4f}")
    # print()

In [402]:
slice_dataframe(df, "salary")

<=50K
<class 'str'>
>50K
<class 'str'>


In [441]:
print(type(X_test))
print(type(X_test[:1]))
# print(X_test[:1])
# print(X_test.shape)
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [446]:
row_slice = test['salary'] == '<=50'
# print(f1_score(y_test[row_slice], lr.predict(X_test[row_slice])))
#     f1_score(y_true,            y_pred)
print(fbeta_score(y_test[row_slice], lr.predict(X_test[row_slice]), beta=1, zero_division=1))

ValueError: Found array with 0 sample(s) (shape=(0, 108)) while a minimum of 1 is required by LogisticRegression.

In [435]:
train[:2]

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
24539,18,Private,120029,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,<=50K
9233,53,Private,68684,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K


In [440]:
# X_train[:2]

In [433]:
y_test[:2]

array([0, 0])

In [444]:
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]

In [445]:
fbeta_score(y_true, y_pred, average='macro', beta=0.5)

0.23809523809523805

In [ ]:
def train_model(X_train, y_train):
#  Using random forest
#     bparameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    rfc = RandomForestClassifier(random_state=42)
    param_grid = {
        'n_estimators': [5, 10]
#         'max_features': ['auto', 'sqrt'],
#         'max_depth': [4, 5, 100],
#         'criterion': ['gini', 'entropy']
    }
    
    cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
    cv_rfc.fit(X_train, y_train)

# using  Support Vector Machines

#     clf = 


    return cv_rfc

In [33]:
from sklearn import svm

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)
GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [1]:
cv_rfc = train_model(X_train, y_train)

NameError: name 'train_model' is not defined

In [457]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7186761229314421, 0.582003828972559, 0.6431593794076164)

In [66]:
rfc = RandomForestClassifier(random_state=42)
# param_grid = {'max_depth': [30, 35, 40], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [2]}
# cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

In [67]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [68]:
preds = inference(rfc, X_test)
compute_model_metrics(y_test, preds)

(0.737593984962406, 0.6264367816091954, 0.6774861878453039)

In [96]:
# rfc = RandomForestClassifier(max_depth=30, min_samples_split=10, min_samples_leaf=2,random_state=42)
I WILL GO WITH THIS ONE 
rfc = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [400, 500, 600], 'max_depth': [100, 200], 'min_samples_split': [2, 10, 50], 'min_samples_leaf': [2], 'criterion': ['gini', 'entropy']}
cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

In [97]:
cv_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [100, 200], 'min_samples_leaf': [2],
                         'min_samples_split': [2, 10, 50],
                         'n_estimators': [400, 500, 600]})

In [99]:
print("Best n_estimators:", cv_rfc.best_params_['n_estimators'])
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best min_samples_leaf:", cv_rfc.best_params_['min_samples_leaf'])
print("Best criterion:", cv_rfc.best_params_['criterion'])
print("Best score:", cv_rfc.best_score_)

Best n_estimators: 600
Best max_depth: 100
Best min_samples_split: 10
Best min_samples_leaf: 2
Best criterion: gini
Best score: 0.8644424219266152


In [100]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7822908204711616, 0.6149425287356322, 0.688594923131927)

In [94]:
print("Best n_estimators:", cv_rfc.best_params_['n_estimators'])
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best min_samples_leaf:", cv_rfc.best_params_['min_samples_leaf'])
print("Best score:", cv_rfc.best_score_)

Best n_estimators: 500
Best max_depth: 100
Best min_samples_split: 2
Best min_samples_leaf: 2
Best score: 0.8643272735178188


In [95]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7811228641171685, 0.6130268199233716, 0.6869409660107334)

In [85]:
print("Best n_estimators:", cv_rfc.best_params_['n_estimators'])
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best min_samples_leaf:", cv_rfc.best_params_['min_samples_leaf'])
print("Best score:", cv_rfc.best_score_)

Best n_estimators: 500
Best max_depth: 100
Best min_samples_split: 2
Best min_samples_leaf: 2
Best score: 0.8643272735178188


In [86]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7811228641171685, 0.6130268199233716, 0.6869409660107334)

In [90]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7811228641171685,
 0.6130268199233716,
 0.6869409660107334,
 0.6869409660107334)

In [80]:
print("Best n_estimators:", cv_rfc.best_params_['n_estimators'])
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best min_samples_leaf:", cv_rfc.best_params_['min_samples_leaf'])
print("Best score:", cv_rfc.best_score_)

Best n_estimators: 400
Best max_depth: 100
Best min_samples_split: 2
Best min_samples_leaf: 2
Best score: 0.8642504833469606


In [81]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.779578606158833, 0.6143039591315453, 0.6871428571428572)

In [63]:
cv_rfc.fit(X_train, y_train)

KeyboardInterrupt: 

In [58]:
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best min_samples_leaf:", cv_rfc.best_params_['min_samples_leaf'])
print("Best score:", cv_rfc.best_score_)

Best max_depth: 30
Best min_samples_split: 10
Best min_samples_leaf: 2
Best score: 0.8636745423265285


In [49]:
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best score:", cv_rfc.best_score_)

Best max_depth: 30
Best min_samples_split: 10
Best score: 0.8641736636981101


In [46]:
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best score:", cv_rfc.best_score_)

Best max_depth: 10
Best min_samples_split: 10
Best score: 0.8572632705316983


In [65]:
preds = inference(rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7825370675453048, 0.6066411238825032, 0.6834532374100719)

In [125]:
rfc = RandomForestClassifier(max_depth=10, min_samples_split=400, min_samples_leaf=2,random_state=42)

In [126]:
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=400,
                       random_state=42)

In [127]:
preds = inference(rfc, X_test)
compute_model_metrics(y_test, preds)

(0.8151093439363817, 0.5236270753512133, 0.6376360808709176)

In [128]:
rfc = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [400, 500, 600], 'max_depth': [4,5 ,100, 200], 'min_samples_split': [2, 10, 50], 'min_samples_leaf': [2]}
cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

In [129]:
cv_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [4, 5, 100, 200], 'min_samples_leaf': [2],
                         'min_samples_split': [2, 10, 50],
                         'n_estimators': [400, 500, 600]})

In [130]:
print("Best n_estimators:", cv_rfc.best_params_['n_estimators'])
print("Best max_depth:", cv_rfc.best_params_['max_depth'])
print("Best min_samples_split:", cv_rfc.best_params_['min_samples_split'])
print("Best min_samples_leaf:", cv_rfc.best_params_['min_samples_leaf'])
print("Best score:", cv_rfc.best_score_)

Best n_estimators: 600
Best max_depth: 100
Best min_samples_split: 10
Best min_samples_leaf: 2
Best score: 0.8644424219266152


In [131]:
preds = inference(cv_rfc, X_test)
compute_model_metrics(y_test, preds)

(0.7822908204711616, 0.6149425287356322, 0.688594923131927)

In [258]:
# I WILL GO WITH THIS ONE IS FASTER AND HIGHER PRECISSION
rfc = RandomForestClassifier(max_depth=10, min_samples_split=400, min_samples_leaf=2, criterion='gini',random_state=42)

In [259]:
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=400,
                       random_state=42)

In [260]:
preds = inference(rfc, X_test)
compute_model_metrics(y_test, preds)

(0.8151093439363817, 0.5236270753512133, 0.6376360808709176)

In [136]:
type(y_test)

numpy.ndarray

In [261]:
y_test[:2]
model = rfc 

In [151]:
# i have a df and getting a unique 
def feature_slice(df, y_test, preds, col):
    '''
    Function to compute model metrics on slices of the dataset
    '''

    for cls in df[col].unique():
        print(cls)

        compute_model_metrics(y_test, preds, cls)
        # df_temp = df[df["class"] == cls]
        # mean = df_temp[col].mean()
        # stddev = df_temp[col].std()
        # print(f"Class: {cls}")
        # print(f"{col} mean: {mean:.4f}")
        # print(f"{col} stddev: {stddev:.4f}")
    # print()


# slice_dataframe(df, "sepal_length")
# slice_iris(df, "sepal_width")
# slice_iris(df, "petal_length")
# slice_iris(df, "petal_width")

In [ ]:
cat_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]

In [265]:
df.head(5)

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [424]:
print(df['workclass'][0])
print(df['workclass'][1])
print(df['workclass'][2])
print(df['workclass'][3])
print(df['workclass'][4])

State-gov
Self-emp-not-inc
Private
Private
Private


In [429]:
for i in df['workclass']:
    print(i)
    aa = i == 'State-gov'

State-gov
Self-emp-not-inc
Private
Private
Private
Private
Private
Self-emp-not-inc
Private
Private
Private
State-gov
Private
Private
Private
Private
Self-emp-not-inc
Private
Private
Self-emp-not-inc
Private
Private
Federal-gov
Private
Private
Local-gov
Private
?
Private
Private
Local-gov
Private
Private
Federal-gov
State-gov
Private
Private
Private
Private
Self-emp-not-inc
Private
Self-emp-not-inc
Private
Private
Private
Federal-gov
Private
Private
State-gov
Private
Private
Private
Private
Federal-gov
Self-emp-inc
Private
Private
Private
Private
Private
Private
?
Private
Private
Private
Private
Private
Private
Self-emp-inc
?
Private
Private
Self-emp-not-inc
Private
Private
Private
Private
?
Private
Local-gov
Private
Private
Private
Private
Private
Private
Local-gov
Private
Private
Federal-gov
Private
Private
Private
Private
Local-gov
Local-gov
Self-emp-not-inc
Private
Private
Federal-gov
Private
Private
Self-emp-not-inc
Private
Private
Self-emp-inc
?
Private
Local-gov
Private
Private


In [428]:
print(type(aa))

<class 'bool'>


In [440]:
slice_d = df[df['workclass'] == 'State-gov']
print(type(slice_d))
# print(slice_d[:8])
# ss = slice_d == True
# print(ss.shape)
print(slice_d.shape)

m = slice_d['workclass'] == 'State-gov'

print(type(m))
print(m.shape)
print(m)

# d = np.where(slice_d)[0]
# print(type(d))
# print(d)

print(y_test.shape)
print(y_test[m])

<class 'pandas.core.frame.DataFrame'>
(1298, 15)
<class 'pandas.core.series.Series'>
(1298,)
0        True
11       True
34       True
48       True
123      True
         ... 
32360    True
32380    True
32479    True
32540    True
32549    True
Name: workclass, Length: 1298, dtype: bool
(6513,)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 6513 but corresponding boolean dimension is 1298

In [446]:
type(X_test)
X_test.shape

(6513, 107)

In [471]:
# row_slice = test[test['workclass'] == 'State-gov']
row_slice = test['workclass'] == 'State-gov'
print(row_slice.shape)
print(type(row_slice))
print(type(row_slice))
print(row_slice)
print(row_slice.value_counts()) 
# m = slice_d['workclass'] == 'State-gov'
# print(type(m))
# print(m.shape)
# print(m)

print(y_test[row_slice].shape)

(6513,)
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
20718    False
28388    False
17643    False
13459    False
24416    False
         ...  
28716    False
29364    False
21590    False
7130     False
5045     False
Name: workclass, Length: 6513, dtype: bool
False    6249
True      264
Name: workclass, dtype: int64
(264,)


In [494]:
def compute_model_metrics(y, preds):
#     print("************--->inside compute_model_metrics ****************")
    precision = precision_score(y, preds, zero_division=1)
    recall = recall_score(y, preds, zero_division=1)
    fbeta = fbeta_score(y, preds, beta=1, zero_division=1)
    print(f"precision: {precision:.4f}")
    print(f"recall: {recall:.4f}")
    print(f"fbeta: {fbeta:.4f}\n")
#     print()
    return precision, recall, fbeta

In [495]:
def get_slice(df, feature):
    print("Performance metrics for " + feature + " slice:\n")
    for cls in df[feature].unique():
        row_slice = df[feature] == cls
        
        preds = inference(model, X_test[row_slice])
        print("Computer metrics for " + cls + " class")
        
        compute_model_metrics(y_test[row_slice], preds)
#         print(x)
        
#     return x

In [496]:
get_slice(test, 'workclass')


Performance metrics for workclass slice:

Computer metrics for Private class
precision: 0.8150
recall: 0.5289
fbeta: 0.6415

Computer metrics for Local-gov class
precision: 0.8036
recall: 0.4737
fbeta: 0.5960

Computer metrics for ? class
precision: 0.8148
recall: 0.4632
fbeta: 0.5906

Computer metrics for Self-emp-not-inc class
precision: 0.8133
recall: 0.5304
fbeta: 0.6421

Computer metrics for Federal-gov class
precision: 0.8276
recall: 0.5714
fbeta: 0.6761

Computer metrics for State-gov class
precision: 0.9020
recall: 0.6479
fbeta: 0.7541

Computer metrics for Self-emp-inc class
precision: 0.6364
recall: 0.3500
fbeta: 0.4516

Computer metrics for Never-worked class
precision: 1.0000
recall: 1.0000
fbeta: 1.0000

Computer metrics for Without-pay class
precision: 1.0000
recall: 1.0000
fbeta: 1.0000



In [459]:
def get_slice(df, feature):
#     print(df[feature])
    count = 0
    # for each categorical column get a slice of dataframe and calculate metrics
    for cls in df[feature].unique():
        print(cls)
    # #     print(df.head(2))
#         sliced_df = df[df[feature] == cls]
        
        sliced_df = df[feature] == cls
        
#         print(y_test.shape)
#         print(type(sliced_df))
#         print(sliced_df.shape)
# #         print(sliced_df[:2])
        
#         print(sliced_df[:3])
        x = sliced_df.iloc[0] == True

        print("....")
        print(x.shape)
        print(type(x))
        print("....")
#         print(sliced_df[:5])
#         print(type(sliced_df))
# #         print(sliced_df[cls])
#         row_slice = sliced_df[feature] == 1
#         print(row_slice.shape)
#         print(type(row_slice))
#         print(row_slice.value_counts())
#         y_s = y_test[row_slice]
#         x_s = X_test[row_slice]
#         print(y_s.shape)
#         print(x_s.shape)
    
        print("-------")
#         print(y_test[cls])
        
#         print(sliced_df.head(2))


        # once you have the df in this case 9 df will happen because there are 9 features in workclss
        # get the metrics of each one
        
        # compute_model_metrics(y_test, preds)
#         if count == 5:
#             break
    #     print(type(df_temp))
    #     print(df_temp.head())
        count+=1
    #     print("**********")
    print(count)
    return sliced_df

In [460]:
# workclass_df = get_slice(df, 'workclass')
workclass_df = get_slice(df, 'workclass')
# print(workclass_df.head(5))
# # workclass_df['workclass'][1]
# # workclass_df.loc['workclass', 'State-gov']
# # df.loc['r4','Duration']
# print("------")
# # print(workclass_df.iloc[0:1])
# print(workclass_df.iloc[0][1])

State-gov
....
()
<class 'numpy.bool_'>
....
-------
Self-emp-not-inc
....
()
<class 'numpy.bool_'>
....
-------
Private
....
()
<class 'numpy.bool_'>
....
-------
Federal-gov
....
()
<class 'numpy.bool_'>
....
-------
Local-gov
....
()
<class 'numpy.bool_'>
....
-------
?
....
()
<class 'numpy.bool_'>
....
-------
Self-emp-inc
....
()
<class 'numpy.bool_'>
....
-------
Without-pay
....
()
<class 'numpy.bool_'>
....
-------
Never-worked
....
()
<class 'numpy.bool_'>
....
-------
9


In [166]:
df['workclass'].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

In [188]:
print(type(y_test))
print(type(preds))
print(y_test[:5])
print(preds[:5])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0 1 0 0 0]
[0 0 0 0 0]


In [205]:
preds.shape

(6513,)

In [346]:
# print(type(train))
# print(type(X_train))
# print(type(y_train))


# print(type(y_test))


# print(train.shape)
# print(X_train.shape)
# print(y_train.shape)
print("----")
print(test.shape)
print(X_test.shape)
print(y_test.shape)
print(type(test))
print(type(X_test))
print(type(y_test))

----
(6513, 15)
(6513, 107)
(6513,)
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [512]:
type(preds)

numpy.ndarray

In [513]:
preds


array([0, 0, 0, ..., 0, 0, 0])

In [514]:
preds[:40]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [515]:
len(preds)

6513

In [516]:
y_test.shape

(6513,)

In [528]:
X_test.shape

(6513, 107)

In [519]:
test.shape

(6513, 15)

In [520]:
train.shape

(26048, 15)

In [521]:
df.shape

(32561, 15)

In [527]:
preds.shape
len(preds)

6513

In [523]:
train.head(5)

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
28687,58,Self-emp-not-inc,105592,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,>50K
20357,35,Private,73715,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
24335,37,Private,167613,HS-grad,9,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,United-States,<=50K
15030,32,Private,193285,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
26772,28,Private,412149,10th,6,Never-married,Farming-fishing,Other-relative,White,Male,0,0,35,Mexico,<=50K


In [524]:
train['salary']

28687     >50K
20357    <=50K
24335    <=50K
15030    <=50K
26772    <=50K
         ...  
23443    <=50K
22588    <=50K
13571    <=50K
8294     <=50K
25501     >50K
Name: salary, Length: 26048, dtype: object

In [537]:
df['marital-status'] = df['marital_status'] 
df['native-country'] = df['native_country'] 

In [538]:
df.head(2)

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary,marital-status,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,Never-married,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,Married-civ-spouse,United-States


In [539]:
print(len(df['workclass'].unique()))
print(len(df['education'].unique()))
print(len(df['marital-status'].unique()))
print(len(df['occupation'].unique()))
print(len(df['relationship'].unique()))
print(len(df['race'].unique()))
print(len(df['sex'].unique()))
print(len(df['native-country'].unique()))

9
16
7
15
6
5
2
42


In [541]:
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [543]:
type(y_test)

numpy.ndarray

In [544]:
len(cat_features)

8

In [545]:
type(preds)

numpy.ndarray

In [546]:
preds.shape

(6513,)

In [547]:
preds[:4]

array([0, 0, 0, 0])